# Random forest importance

In [1]:
%store -r X_256
%store -r y_256
%store -r X_test_256
%store -r y_test_256
%store -r feature_names_256

In [2]:
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
import matplotlib.pyplot as plt

# Reshape data
# Data ban đầu X(13166, 20, 128), y(13166)

# Reshape X về dạng 2d => X_2d(263320, 128), giữ nguyên 128 và đó là các đặc trưng
X_2d = X_256.reshape(-1, X_256.shape[-1])
# Reshape y thành y_reshaped(263320) bằng cách lặp mỗi phần tử trong y 20 lần
y_reshaped = np.repeat(y_256, 20)

In [4]:
# Khởi tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=500)
# Nó chỉ định số lượng cây quyết định sẽ được xây dựng và kết hợp để tạo thành khu rừng ngẫu nhiên trong thuật toán random forest.

In [5]:
# Huấn luyện mô hình
model.fit(X_2d, y_reshaped)

RandomForestClassifier(n_estimators=500)

In [6]:
# Lấy tầm quan trọng của đặc trưng
feature_importances = model.feature_importances_

# In tầm quan trọng của đặc trưng
print(feature_importances)

[0.01494289 0.01194984 0.01188773 0.01118555 0.01141443 0.01167344
 0.01132301 0.01161113 0.01140211 0.01135212 0.01117699 0.01116034
 0.01812214 0.02479548 0.02844017 0.02768187 0.0215653  0.03095806
 0.02118948 0.0224589  0.01371308 0.01694898 0.014569   0.01700924
 0.01367944 0.01265524 0.01321888 0.01415732 0.01007837 0.0073284
 0.00663491 0.00425329 0.00330146 0.00307803 0.00317295 0.00322657
 0.00264336 0.00306234 0.00273828 0.00393715 0.00329305 0.00282089
 0.003414   0.00327893 0.0028771  0.00303598 0.00290151 0.00260423
 0.00290207 0.0031785  0.00273609 0.00265002 0.00305743 0.0030694
 0.00345518 0.00291254 0.00341511 0.0038965  0.00417727 0.00445708
 0.00532054 0.00505492 0.00546833 0.00430156 0.00324031 0.00332484
 0.00416518 0.00320879 0.00258856 0.002694   0.00364946 0.00355745
 0.00328331 0.00356278 0.00367486 0.00273047 0.00304484 0.00267591
 0.00388926 0.00378224 0.00341322 0.00367835 0.00340943 0.00354349
 0.00296555 0.00315527 0.00225058 0.00219898 0.00187356 0.002194

In [7]:
feature_importances.shape

(256,)

In [8]:
feature_names_256

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '246', '247', '248', '249', '250', '251', '252', '253', '254', '255'],
      dtype='object', length=256)

In [9]:
# Tạo dataframe gồm tên đặc trưng và tầm quan trọng
feature_importances_df = pd.DataFrame({"feature_name": feature_names_256, "importance": feature_importances})

# Sắp xếp dataframe theo tầm quan trọng
feature_importances_df = feature_importances_df.sort_values("importance", ascending=False)

# Lấy index của dataframe
indices = feature_importances_df.index

# In index
print(indices)

Index([ 17,  14,  15,  13,  19,  16,  18,  12,  23,  21,
       ...
       201, 200, 226, 197, 211, 204, 198, 190, 196, 212],
      dtype='int64', length=256)


In [10]:
# Xác định số đặc trưng sẽ giữ lại để train  model
k = 128
top_k_indices = indices[:k]

In [11]:
# Giữ lại k đặc trưng tốt nhất trong tệp dữ liệu ban đầu
selected_X = X_256[:,:,top_k_indices]
selected_X_test = X_test_256[:,:,top_k_indices]

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

model.add(layers.Input(shape = (20, k, 1)))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 20, 128, 32)    │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 20, 128, 32)    │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 10, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 64, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 64, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 32, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10240)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       655,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 835,941 (3.19 MB)

 Trainable params: 835,941 (3.19 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(selected_X, y_256, epochs=10, batch_size=16)

Epoch 1/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 71s 86ms/step - accuracy: 0.5385 - loss: 1.0997
Epoch 2/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 67s 81ms/step - accuracy: 0.7959 - loss: 0.5350
Epoch 3/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 65s 79ms/step - accuracy: 0.8297 - loss: 0.4566
Epoch 4/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 64s 78ms/step - accuracy: 0.8429 - loss: 0.4034
Epoch 5/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 65s 79ms/step - accuracy: 0.8427 - loss: 0.3820
Epoch 6/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 66s 81ms/step - accuracy: 0.8585 - loss: 0.3600
Epoch 7/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 68s 83ms/step - accuracy: 0.8757 - loss: 0.3316
Epoch 8/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 64s 77ms/step - accuracy: 0.8923 - loss: 0.2899
Epoch 9/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 73s 89ms/step - accuracy: 0.8959 - loss: 0.2705
Epoch 10/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 63s 76ms/step - accuracy: 0.8906 - loss: 0.2748


In [14]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test_256)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test_256, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test_256, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.8997 - loss: 0.2232
Độ chính xác trên tập kiểm tra: 0.9144007563591003
412/412 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1990   32    3    9]
 [   0   22 2730    0   56]
 [   0  348   26 1364    2]
 [   0   34  595    0 3113]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.83      0.98      0.90      2034
           2       0.81      0.97      0.88      2808
           3       1.00      0.78      0.88      1740
           4       0.98      0.83      0.90      3742

    accuracy                           0.91     13166
   macro avg       0.92      0.91      0.91     13166
weighted avg       0.93      0.91      0.91     13166



In [18]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def micro_metrics(y_true, y_pred):
  # Calculate micro averages
  micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
      y_true, y_pred.argmax(axis=1), average='micro'
  )
    
  # Macro averages
  macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
      y_true, y_pred.argmax(axis=1), average='macro'
  )
  return {
      'micro_precision': micro_precision,
      'micro_recall': micro_recall,
      'micro_f1': micro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'macro_f1': macro_f1
  }

In [19]:
from tabulate import tabulate
# Prepare table data
table_data = [
  ["Micro Precision", micro_results['micro_precision']],
  ["Micro Recall", micro_results['micro_recall']],
  ["Micro F1-score", micro_results['micro_f1']],
  ["Macro Precision", micro_results['macro_precision']],
  ["Macro Recall", micro_results['macro_recall']],
  ["Macro F1-score", micro_results['macro_f1']],
]

# Print the table using tabulate
print(tabulate(table_data, headers=["Metric", "Score"]))

Metric              Score
---------------  --------
Micro Precision  0.92115
Micro Recall     0.92115
Micro F1-score   0.92115
Macro Precision  0.925261
Macro Recall     0.92604
Macro F1-score   0.925651
